In [130]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import pandas as pd

In [88]:
driver = webdriver.Chrome()
driver.get('https://help.shopee.co.id/portal/4/category/19-Belanja-di-Shopee/378-Baru-di-Shopee?page=1')

driver.execute_script('window.scrollTo(0, 200)')
time.sleep(0.5)

try:
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'header___rvuhG'))
    )

    dropdowns = driver.find_elements(By.CLASS_NAME, 'header___rvuhG')
    dropdowns[0].click()
    time.sleep(1)
    
    questions = []

    for i, dropdown in enumerate(dropdowns):
        try:
            dropdown.click()
            time.sleep(1)

        except Exception as e:
            pass

except Exception as e:
    pass

driver.execute_script('window.scrollTo(0, 300);')
time.sleep(0.5)

questions = []

try:
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'category_item___LqcUb'))
    )
    
    dropdown_items = driver.find_elements(By.CLASS_NAME, 'category_item___LqcUb')
    dropdown_items[1].click()
    time.sleep(1)

    for i, dropdown_item in enumerate(dropdown_items):
        try:
            dropdown_item.click()
            time.sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            questions += soup.find('div', class_='list___RQ7Sh').find_all('a')

            paginations = driver.find_elements(By.CLASS_NAME, 'shopee-react-pagination-pager__page')
            if len(paginations):
                # for j in range(int(paginations[-1].text)):
                #     current_url = driver.current_url
                #     new_url = current_url.split('?')[0]
                #     new_url += f'?page={j+1}'
                #     driver.get(new_url)
                #     time.sleep(2)
                for j, pagination in enumerate(paginations[1:], start=2):
                    WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, 'shopee-react-pagination-pager__page'))
                    )
                    pagination.click()
                    time.sleep(1)
                    questions += soup.find('div', class_='list___RQ7Sh').find_all('a')

        except Exception as e:
            pass

except Exception as e:
    pass 

driver.quit()

In [ ]:
questions_text = []
answers_text = []

driver = webdriver.Chrome()

for question in questions:
    questions_text.append(question.text)
    driver.get(question.get('href'))
    time.sleep(1.25)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    answers_text.append(soup.find('div', class_='right___P29Sy').text)
    
driver.quit()

print(f'Panjang list pertanyaan: {len(questions_text)}')
print(f'Panjang list jawaban: {len(answers_text)}')

df = pd.DataFrame({
    'questions': questions_text,
    'answers': answers_text
})

df.head()

df.to_csv('shopee_qna_data_raw.csv', index=False)

Panjang list pertanyaan: 937
Panjang list jawaban: 937


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  937 non-null    object
 1   answers    937 non-null    object
dtypes: object(2)
memory usage: 14.8+ KB


In [143]:
df.set_index('questions', inplace=True)

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 937 entries, [Shopee Live] Apa itu Lelang di Shopee Live? to Bagaimana proses pengembalian dana secara manual jika status pesanan gagal di sisi Merchant ShopeePay, namun status pembayaran ShopeePay/SPayLater berhasil?
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   answers  937 non-null    object
dtypes: object(1)
memory usage: 14.6+ KB


In [147]:
df.drop_duplicates(inplace=True)

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257 entries, [Shopee Live] Apa itu Lelang di Shopee Live? to Bagaimana proses pengembalian dana secara manual jika status pesanan gagal di sisi Merchant ShopeePay, namun status pembayaran ShopeePay/SPayLater berhasil?
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   answers  257 non-null    object
dtypes: object(1)
memory usage: 4.0+ KB


In [149]:
df

,answers
questions,
[Shopee Live] Apa itu Lelang di Shopee Live?,[Shopee Live] Apa itu Lelang di Shopee Live?Ak...
[Grosir] Apa itu Produk Grosir?,[Grosir] Apa itu Produk Grosir?Produk Grosir m...
[Shopee Barokah] Apa itu Shopee Barokah?,[Shopee Barokah] Apa itu Shopee Barokah?Shopee...
[Baru di Shopee] Siapa Penjual Star atau Star+ Shopee?,[Baru di Shopee] Siapa Penjual Star atau Star+...
[Shopee Live] Apa saja kategori streaming yang dianggap berkualitas rendah di Shopee Live?,[Shopee Live] Apa saja kategori streaming yang...
...,...
Bagaimana cara Merchant ShopeePay menyimpan atau membagikan Kode QRIS outlet?,Bagaimana cara Merchant ShopeePay menyimpan at...
Berapa biaya dan saldo minimum penarikan dana Merchant ShopeeFood/ShopeePay?,Berapa biaya dan saldo minimum penarikan dana ...
Apa saja kriteria untuk melakukan pengembalian dana transaksi Merchant ShopeePay?,Apa saja kriteria untuk melakukan pengembalian...
